# Forecasting

In order to generate models, quality checks, reserves, etc., we need forecasts of the well production. This notebook demonstrates:

1. **Data Processing**: Preparing well production data for forecasting
2. **ARPS Decline Curves**: Automatically fitting exponential, hyperbolic, and harmonic decline curves
3. **Forecasting**: Generating production forecasts for individual wells

## ARPS Decline Curves

The Arps decline curve equations are fundamental tools in petroleum engineering for forecasting oil and gas production:

- **Exponential Decline (b=0)**: `q(t) = qi * exp(-Di * t)`
- **Hyperbolic Decline (0<b<1)**: `q(t) = qi * (1 + b * Di * t)^(-1/b)`
- **Harmonic Decline (b=1)**: `q(t) = qi / (1 + Di * t)`

Where:
- `qi` = initial production rate
- `Di` = initial decline rate
- `b` = decline exponent
- `t` = time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from petrinex.config import DotConfig
from petrinex.forecast import (
    prepare_well_data,
    forecast_well_production,
    forecast_multiple_wells,
    export_forecast_summary_table,
    export_forecasted_production_table,
    combine_historical_and_forecast
)

In [ ]:
config = DotConfig("local_config.yaml")

In [ ]:
# Load fixture data for development/testing
ngl_df = pd.read_parquet("../fixtures/ngl_vol_silver.parquet")

In [ ]:
# Prepare well data for forecasting
processed_data = prepare_well_data(ngl_df, min_months=6)
sample_well = processed_data['WellID'].iloc[0] if len(processed_data) > 0 else None

In [ ]:
# Forecast a single well
if sample_well:
    sample_well_data = processed_data[processed_data['WellID'] == sample_well].copy()
    
    # Forecast gas production
    gas_forecast = forecast_well_production(
        sample_well_data, 
        forecast_months=12, 
        production_column='GasProduction'
    )

In [ ]:
# Optional: Plot the forecast
if sample_well and gas_forecast['success']:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Historical data
    hist_data = gas_forecast['historical_data']
    hist_data_filtered = hist_data[hist_data['GasProduction'] > 0]
    
    ax.scatter(hist_data_filtered['DaysFromFirst'], hist_data_filtered['GasProduction'],
               alpha=0.7, label='Historical', color='blue')
    
    # Forecast
    forecast_data = gas_forecast['forecast']
    ax.plot(forecast_data['DaysFromFirst'], forecast_data['GasProduction_Forecast'],
            'r-', label='Forecast', linewidth=2)
    
    ax.set_xlabel('Days from First Production')
    ax.set_ylabel('Gas Production')
    ax.set_title(f'{sample_well} Gas Forecast')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()


In [ ]:
# Batch forecast multiple wells
gas_forecasts = forecast_multiple_wells(
    processed_data,
    forecast_months=12,
    production_column='GasProduction', 
    curve_type='auto',
    min_r_squared=0.5
)


In [ ]:
# Export forecast results
gas_summary_table = export_forecast_summary_table(gas_forecasts, "GasProduction")
gas_forecast_production = export_forecasted_production_table(gas_forecasts, "GasProduction", processed_data)


In [ ]:
# Save results to files
if len(gas_summary_table) > 0:
    gas_summary_table.to_parquet("../fixtures/gas_forecast_summary.parquet", index=False)

if len(gas_forecast_production) > 0:
    gas_forecast_production.to_parquet("../fixtures/gas_forecasted_production.parquet", index=False)
    
    # Combine historical and forecast data
    combined_gas_data = combine_historical_and_forecast(
        processed_data, gas_forecast_production, "GasProduction"
    )
    combined_gas_data.to_parquet("../fixtures/gas_historical_forecast_combined.parquet", index=False)


In [ ]:
print(f"Forecasting complete. Generated forecasts for {len(gas_forecasts)} wells.")
